# A Test for the Graph Explorer Widget

This uses [importnb](https://pypi.org/project/importnb) to load the
[GraphExploreSelectMultiple](./GraphExploreSelectMultiple.ipynb) for interactive and
automated testing.

Current implementation tests the ipyradiant widgets only (not RDF2NX code in notebook).

In [ ]:
import json
import os
import time
from pathlib import Path

import importnb
import ipywidgets as W
from rdflib import Graph
from requests_cache import CachedSession

with importnb.Notebook():
    try:
        from GraphExploreSelectMultiple import (
            LEN_SSSW_VALUES,
            LEN_TSSW_VALUES,
            SSSW_VALUES,
            TSSW_VALUES,
            ge_selector,
            lw,
        )
    except GraphExploreSelectMultiple:
        from .GraphExplorer import (
            LEN_SSSW_VALUES,
            LEN_TSSW_VALUES,
            SSSW_VALUES,
            TSSW_VALUES,
            ge_selector,
            lw,
        )

In [ ]:
fp = lw.loader.file_picker
tssw = ge_selector.type_select.select_widget
sssw = ge_selector.subject_select.select_widget

This notebook is used as part of the `ipyradiant` test suite, where `IPYRADIANT_TESTING`
will be set, which will trigger the automted section below.

In [ ]:
IS_TESTING = json.loads(os.environ.get("IPYRADIANT_TESTING", "false"))
IS_TESTING

Tests are implemented as buttons you can click. Only using `starwars` graphs for
testing.

In [ ]:
tests = [
    W.Button(description=f"{k}") for k, v in fp.options.items() if v and "starwars" in k
]

In [ ]:
timings = {}


def timestamp(key, msg):
    if key not in timings:
        timings[key] = []
    timings[key] += [time.time()]
    delta = 0 if len(timings[key]) == 1 else timings[key][-1] - timings[key][-2]
    print(f"[{key}]", f"+{int(delta)}", msg)

In [ ]:
def _run_test(btn):
    p = btn.description

    try:
        timestamp(p, "starting...")
        ge_selector.graph = Graph()
        timestamp(p, "cleaned...")
        assert not tssw.value
        assert not sssw.value
        timestamp(p, f"loading...")
        fp.value = fp.options[p]
        timestamp(p, f"... {len(lw.graph)} triples loaded")
        ge_selector.graph = lw.graph
        assert len(ge_selector.graph), "Graph failed to populate."
        assert len(tssw.options), "Type Select options failed to populate."
        assert len(sssw.options) == 0, f"{len(sssw.options)} != 0"
        tssw.value = TSSW_VALUES
        timestamp(p, f"type selections made...")
        assert len(sssw.options), "Failed to populate Subject Select options."
        sssw.value = SSSW_VALUES
        timestamp(p, f"subject selections made...")
        assert len(sssw.value) == LEN_SSSW_VALUES
        timestamp(p, "OK!")
    except Exception as err:
        timestamp(p, "ERROR")
        timestamp(p, err)
        raise Exception(f"{p} failed") from err


[d.on_click(_run_test) for d in tests]

## Show the Test Application

In [ ]:
W.VBox([ge_selector, W.HBox([W.Label("Start Test"), *tests])])

## (Maybe) Run the test automatically

In [ ]:
if IS_TESTING:
    for test in tests:
        test.click()